In [1]:
import mlflow
import pandas as pd

In [2]:
import sys
sys.path.append('../../src')

from search import search

In [3]:
mlflow.set_tracking_uri("../../mlruns")
mlflow.set_experiment("graph_expansion")

<Experiment: artifact_location='/home/felix/Documents/lexum-test/tmp/notebooks/03_experiments/../../mlruns/286683048842659356', creation_time=1747147672863, experiment_id='286683048842659356', last_update_time=1747147672863, lifecycle_stage='active', name='graph_expansion', tags={}>

In [4]:
df = pd.read_parquet('../../data/retrieval_questions_multichunks.parquet')

df['target'] = df.target.apply(lambda x: x.tolist())
df['entities'] = df.entities.apply(
    lambda x: {
        k: [v_ for v_ in v if v_ is not None]
        for k, v in x.items()
        if v is not None and len(v) > 0
    }
)

In [5]:
df['retrieved'] = df.apply(
    search,
    method="bm25",
    k=3,
    filter_by_entity=False,
    do_rerank=False,
    do_graph_expansion=True,
    sleep=5,
    axis=1
)

2025-05-13T14:57:18.887 [BAML INFO] Function ExtractTriples:
    Client: GeminiFlash (gemini-2.0-flash) - 608ms. StopReason: STOP. Tokens(in/out): 1530/22
    ---PROMPT---
    user: Your task is to find facts that help answer an input question.
    You should present these facts as knowlege triples, which are structured as ("subject", "predicate", "object").
    Example:
    Question: When was Neville A. Stanton’s employer founded?
    Facts: ("Neville A. Stanton", "employer", "University of Southampton"), ("University of Southampton", "founded in",
    "1862")
    Now you are given some documents:
    Document 0:
    # Sonic Forces
    
    Sonic Forces is a 2017 platform game developed by Sonic Team and published by Sega. It was produced in commemoration of the 25th anniversary of the Sonic the Hedgehog franchise. The plot focuses on Sonic the Hedgehog joining a resistance movement formed by his friends, alongside its rookie and his misplaced past self. Together, they must stop Docto

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

2025-05-13T14:59:22.738 [BAML INFO] Function ExtractTriples:
    Client: GeminiFlash (gemini-2.0-flash) - 1291ms. StopReason: STOP. Tokens(in/out): 944/106
    ---PROMPT---
    user: Your task is to find facts that help answer an input question.
    You should present these facts as knowlege triples, which are structured as ("subject", "predicate", "object").
    Example:
    Question: When was Neville A. Stanton’s employer founded?
    Facts: ("Neville A. Stanton", "employer", "University of Southampton"), ("University of Southampton", "founded in",
    "1862")
    Now you are given some documents:
    Document 0:
    # Star Wars: Battlefront
    
    Star Wars: Battlefront is a series of first- and third-person shooter video games based on the Star Wars franchise. Players take the role of characters from the franchise in either of two opposing factions in different time periods of the Star Wars universe. The series was launched in 2004 by LucasArts with Star Wars: Battlefront, develo

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

2025-05-13T15:02:53.252 [BAML INFO] Function ExtractTriples:
    Client: GeminiFlash (gemini-2.0-flash) - 827ms. StopReason: STOP. Tokens(in/out): 1188/82
    ---PROMPT---
    user: Your task is to find facts that help answer an input question.
    You should present these facts as knowlege triples, which are structured as ("subject", "predicate", "object").
    Example:
    Question: When was Neville A. Stanton’s employer founded?
    Facts: ("Neville A. Stanton", "employer", "University of Southampton"), ("University of Southampton", "founded in",
    "1862")
    Now you are given some documents:
    Document 0:
    # Freedom Planet
    
    Freedom Planet is a 2014 platform game developed and published by American video game designer Sabrina DiDuro, for her studio GalaxyTrail. The player controls one of three anthropomorphic animal protagonists: the dragon girl Lilac, the wildcat Carol, or the basset hound Milla. Aided by a duck-like alien named Torque, the girls attempt to defeat t

In [6]:
df.retrieved.apply(len).describe()

count    94.000000
mean      4.574468
std       0.933134
min       3.000000
25%       4.000000
50%       5.000000
75%       5.000000
max       6.000000
Name: retrieved, dtype: float64

In [7]:
docs_retrieved_baseline = []

for _, row in df.iterrows():
    k = len(row.retrieved)
    docs = search(
        row,
        method="bm25",
        k=k,
        filter_by_entity=False,
        do_rerank=False,
        do_graph_expansion=False,
    )

    docs_retrieved_baseline.append(docs)

df['retrieved_baseline'] = docs_retrieved_baseline

In [8]:
assert (df['retrieved'].apply(len) == df['retrieved_baseline'].apply(len)).all()

In [9]:
df['retrieved_id'] = df.retrieved.apply(lambda x: [d.id for d in x])

df['retrieved_baseline_id'] = df.retrieved_baseline.apply(
    lambda x: [d.id for d in x]
)

In [11]:
recall_baseline = df.apply(lambda x:
    sum(
        [1 if d in x.retrieved_baseline_id else 0 for d in x.target]
    ) / len(x.target),
    axis=1
)

recall_baseline.mean()

np.float64(0.5638297872340425)

In [12]:
recall = df.apply(lambda x:
    sum(
        [1 if d in x.retrieved_id else 0 for d in x.target]
    ) / len(x.target),
    axis=1
)

recall.mean()

np.float64(0.48404255319148937)

In [26]:
with mlflow.start_run():
    mlflow.log_param("method", "bm25")
    mlflow.log_param("k", 3)
    mlflow.log_param("filter_by_entity", False)
    mlflow.log_param("do_rerank", False)
    mlflow.log_param("do_graph_expansion", True)
    mlflow.log_metric("recall", recall.mean())

In [27]:
with mlflow.start_run():
    mlflow.log_param("method", "bm25")
    mlflow.log_param("k", 3)
    mlflow.log_param("filter_by_entity", False)
    mlflow.log_param("do_rerank", False)
    mlflow.log_param("do_graph_expansion", False)
    mlflow.log_metric("recall", recall_baseline.mean())